# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.utils.horizons import (
    horizons_bulk_vector_query,
    horizons_bulk_astrometry_query,
    old_horizons_bulk_vector_query,
)

t0 = Time("2024-12-24T00:00:00", scale="utc")

In [2]:
t0.tdb.jd

np.float64(2460668.500800737)

In [9]:
horizons_bulk_vector_query("274301", "500@0", t0)

,JDTDB,x,y,z,vx,vy,vz,LT,RG,RR
0,2.460669e+06,-1.94951,1.832663,0.536383,-0.006898,-0.00641,-0.001975,0.015761,2.72891,0.000235


In [10]:
horizons_obj = Horizons(id="274301", location="@0", epochs=[t0.tdb.jd])
horizons_obj.vectors(refplane="earth")

targetname,datetime_jd,datetime_str,H,G,x,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,mag,---,AU,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str28,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
274301 Wikipedia (2008 QH24),2460668.500800737,A.D. 2024-Dec-24 00:01:09.1837,16.98,0.15,-1.949509855309397,1.832663475866909,0.536382607852957,-0.006898065162925186,-0.006410241473647372,-0.001974923012382411,0.0157608703707276,2.728910110965948,0.0002347883278320848


In [11]:
old_horizons_bulk_vector_query("274301", "500@0", t0.tdb.jd)

,JDTDB,Cal,x,y,z,vx,vy,vz,LT,RG,RR,_
0,2.460669e+06,A.D. 2024-Dec-24 00:01:09.1837,-1.94951,1.832663,0.536383,-0.006898,-0.00641,-0.001975,0.015761,2.72891,0.000235,NaN


In [12]:
large_times = Time(
    jnp.linspace(t0.tdb.jd, t0.tdb.jd + 365, 100), format="jd", scale="tdb"
)

jorb = horizons_bulk_vector_query("274301", "500@0", large_times)

In [14]:
jorb.iloc[-25:]

,JDTDB,Cal,x,y,z,vx,vy,vz,LT,RG,RR
75,2.460945e+06,A.D. 2025-Sep-26 12:22:58.2746,-2.506629,-0.562438,-0.187139,0.003588,-0.009247,-0.002753,0.014876,2.575761,-0.001273
76,2.460949e+06,A.D. 2025-Sep-30 04:52:03.7292,-2.493103,-0.596465,-0.197266,0.003749,-0.009210,-0.002741,0.014849,2.571040,-0.001288
77,2.460952e+06,A.D. 2025-Oct-03 21:21:09.1837,-2.478986,-0.630350,-0.207346,0.003910,-0.009171,-0.002728,0.014821,2.566263,-0.001304
78,2.460956e+06,A.D. 2025-Oct-07 13:50:14.6382,-2.464275,-0.664086,-0.217378,0.004070,-0.009129,-0.002714,0.014794,2.561428,-0.001319
79,2.460960e+06,A.D. 2025-Oct-11 06:19:20.0928,-2.448972,-0.697663,-0.227357,0.004231,-0.009085,-0.002699,0.014765,2.556539,-0.001334
80,2.460963e+06,A.D. 2025-Oct-14 22:48:25.5473,-2.433077,-0.731073,-0.237281,0.004392,-0.009038,-0.002684,0.014737,2.551594,-0.001348
81,2.460967e+06,A.D. 2025-Oct-18 15:17:31.0019,-2.416590,-0.764305,-0.247149,0.004552,-0.008989,-0.002668,0.014708,2.546596,-0.001363
82,2.460971e+06,A.D. 2025-Oct-22 07:46:36.4564,-2.399512,-0.797351,-0.256956,0.004712,-0.008937,-0.002652,0.014679,2.541545,-0.001377
83,2.460975e+06,A.D. 2025-Oct-26 00:15:41.9110,-2.381843,-0.830202,-0.266700,0.004872,-0.008883,-0.002634,0.014649,2.536443,-0.001391
84,2.460978e+06,A.D. 2025-Oct-29 16:44:47.3655,-2.363584,-0.862848,-0.276379,0.005032,-0.008826,-0.002616,0.014620,2.531289,-0.001405


In [16]:
horizons_obj = Horizons(id="274301", location="@0", epochs=large_times.tdb.jd[-25:])
horizons_obj.vectors(refplane="earth")

targetname,datetime_jd,datetime_str,H,G,x,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,mag,---,AU,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str28,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
274301 Wikipedia (2008 QH24),2460945.015952252,A.D. 2025-Sep-26 12:22:58.2746,16.98,0.15,-2.506628687257302,-0.5624381101068597,-0.1871391885578337,0.003588124572116298,-0.009247435334208125,-0.002752802970084207,0.01487635526558127,2.575761067990571,-0.00127256703265886
274301 Wikipedia (2008 QH24),2460948.702820939,A.D. 2025-Sep-30 04:52:03.7292,16.98,0.15,-2.493103449583946,-0.5964647009664481,-0.1972660414429455,0.003748857703568006,-0.009210425530057033,-0.002740547915903787,0.01484909107619264,2.571040419933715,-0.001288186408417443
274301 Wikipedia (2008 QH24),2460952.389689626,A.D. 2025-Oct-03 21:21:09.1837,16.98,0.15,-2.478985584251678,-0.6303504121034265,-0.2073463808670489,0.003909599635457207,-0.009171009648072981,-0.002727570066948052,0.01482149664284442,2.566262591907784,-0.001303584307401994
274301 Wikipedia (2008 QH24),2460956.076558313,A.D. 2025-Oct-07 13:50:14.6382,16.98,0.15,-2.464275116230339,-0.6640863365228646,-0.217377531539842,0.004070319087717217,-0.009129168100015744,-0.002713863759093967,0.01479357673942759,2.561428410486376,-0.001318755279367485
274301 Wikipedia (2008 QH24),2460959.763427,A.D. 2025-Oct-11 06:19:20.0928,16.98,0.15,-2.448972187299044,-0.6976634950000113,-0.2273567972955692,0.004230983971693172,-0.009084881294724848,-0.002699423333118574,0.01476533625688878,2.556538722510651,-0.001333693779899194
274301 Wikipedia (2008 QH24),2460963.450295687,A.D. 2025-Oct-14 22:48:25.5473,16.98,0.15,-2.433077059073071,-0.7310728360881417,-0.2372814611170474,0.00439156136855851,-0.00903812965521565,-0.002684243140011402,0.01473678020525861,2.551594395440518,-0.001348394169258667
274301 Wikipedia (2008 QH24),2460967.137164373,A.D. 2025-Oct-18 15:17:31.0019,16.98,0.15,-2.416590116092756,-0.76430523622909,-0.2471487851910569,0.004552017507746415,-0.008988893636713471,-0.002668317546561469,0.01470791371567195,2.546596317704442,-0.001362850711276676
274301 Wikipedia (2008 QH24),2460970.824033061,A.D. 2025-Oct-22 07:46:36.4564,16.98,0.15,-2.399511869009606,-0.7973514998941914,-0.256956010973599,0.004712317745060341,-0.008937153745760039,-0.002651640941260351,0.01467874204244345,2.541545399058799,-0.001377057572273019
